In [1]:
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification, pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
import json, os
import random
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
max_length = 256

premise = "Two women are embracing while holding to go packages."
hypothesis = "The men are fighting outside a deli."

# hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)

tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                    max_length=max_length,
                                                    return_token_type_ids=True, truncation=True)

input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
# remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

outputs = model(input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=None)
# Note:
# "id2label": {
#     "0": "entailment",
#     "1": "neutral",
#     "2": "contradiction"
# },
print(outputs[0])
predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

print("Premise:", premise)
print("Hypothesis:", hypothesis)
print("Entailment:", predicted_probability[0])
print("Neutral:", predicted_probability[1])
print("Contradiction:", predicted_probability[2])

tensor([[-4.1005, -1.4663,  5.8797]], grad_fn=<AddmmBackward0>)
Premise: Two women are embracing while holding to go packages.
Hypothesis: The men are fighting outside a deli.
Entailment: 4.627431553672068e-05
Neutral: 0.0006447185878641903
Contradiction: 0.9993090629577637


In [7]:
processed_data = []

with open('flan-t5/processed.json', 'r') as f:
    flant5 = json.load(f)

with open('flan-t5-prompted/processed.json', 'r') as f:
    flant5prompted = json.load(f)

with open('bloom/processed.json', 'r') as f:
    bloom = json.load(f)

with open('gpt-3.5/processed.json', 'r') as f:
    gpt = json.load(f)

processed_data += flant5 + flant5prompted + bloom + gpt
random.shuffle(processed_data)

In [5]:
processed_data[0]

{'premise': 'A young boy with black hair is holding onto a gun which is secured to a vehicle which is being driven by a young man who is wearing a bright pink shirt.',
 'hypothesis': 'The boy is outside and running from cops.',
 'uid': '4762813376.jpg#0r1n',
 'explanation': 'The boy cannot be running from cops and holding onto a gun at the same time.',
 'generation_label': 'contradiction',
 'fluency': True,
 '_id': 'main_issue_3',
 'explanation_only': 'contradiction',
 'explanation_with_premise': 'neutral',
 'consistency': False,
 'majority_label': 'n',
 'label_counter': {'n': 73, 'c': 27, 'e': 0},
 'label_rank': {'n': 'high', 'e': 'mid', 'c': 'low'}}

In [13]:
classifier = pipeline('zero-shot-classification',model=hg_model_hub_name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:

candidate_labels = ['grammatical', 'not grammatical']

counter1 = 0
counter2 = 0
for i in tqdm(range(len(processed_data)), desc='zero-shot-classification'):
    out = classifier(processed_data[i]['explanation'], candidate_labels)
    label_idx = torch.argmax(torch.tensor(out['scores'])).item()
    label = candidate_labels[label_idx]
    if label == 'grammatical' and processed_data[i]['fluency'] == True:
        counter1 += 1
    elif label == 'not grammatical' and processed_data[i]['fluency'] == False:
        counter2 += 1

zero-shot-classification: 100%|██████████| 400/400 [02:37<00:00,  2.54it/s]


In [8]:
proper_data = []
improper_data = []
for d in processed_data:
    if d['fluency'] == True and d['consistency'] == False:
        if d['explanation_with_premise'] == d['generation_label']:
            proper_data.append(d)
        else:
            improper_data.append(d)

In [9]:
def get_probs(model, tokenizer, premise, hypothesis):
    tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                    max_length=1024,
                                                    return_token_type_ids=True, truncation=True)

    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

    outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    
    return torch.softmax(outputs[0], dim=1)[0]

In [10]:
shifted = 0
changed = 0
counter = 0
prev_probs = []
after_probs = []
target_probs = []


labels = ['entailment', 'neutral', 'contradiction']
for i in tqdm(proper_data, desc='proper_data'):
    if not 'ibm' in i['uid']:
        probs = get_probs(model, tokenizer, i['premise'], i['hypothesis'])
        prev_probs.append(torch.log(probs))
        e_probs = get_probs(model, tokenizer, i['premise'] + i['explanation'], i['hypothesis'])
        after_probs.append(torch.log(e_probs))

        target_prob = [i['label_counter'][x] for x in ['e', 'n', 'c']]
        target_probs.append(torch.tensor(target_prob)/100)

        prev_label = labels[torch.argmax(probs).item()]
        e_label = labels[torch.argmax(e_probs).item()]

        if prev_label != i['generation_label']:
            counter += 1
            if e_label == i['generation_label']:
                changed += 1
            
            label_idx = labels.index(i['generation_label'])

            if e_probs.tolist()[label_idx] > probs.tolist()[label_idx]:
                shifted += 1

proper_data:  93%|█████████▎| 178/191 [02:00<00:13,  1.04s/it]

: 

: 

In [ ]:
print(changed/counter, shifted/counter, counter)

In [56]:
prev_probs = torch.stack(prev_probs, dim = 0)
after_probs = torch.stack(after_probs, dim = 0)
target_probs = torch.stack(target_probs, dim = 0)

kl_loss = torch.nn.KLDivLoss(reduction="batchmean")

print(kl_loss(prev_probs, target_probs), kl_loss(after_probs, target_probs))


tensor(0.3147, grad_fn=<DivBackward0>) tensor(1.3756, grad_fn=<DivBackward0>)


In [57]:
prev_probs.shape

torch.Size([187, 3])

In [51]:
after_probs

tensor([[-5.8367e+00, -4.9098e+00, -3.5938e+00, -2.5398e-02, -5.4404e+00,
         -6.1269e+00, -5.8367e+00, -5.4111e+00, -3.6748e-01, -3.8139e+00,
         -4.3025e+00, -1.2362e-02, -9.3099e-01, -3.7105e-02, -9.3099e-01,
         -3.5938e+00, -5.8367e+00, -5.4111e+00, -5.7795e+00, -5.1388e-01,
         -7.4089e+00, -7.4441e-02, -5.4404e+00, -6.4468e+00, -3.0649e-01,
         -4.3025e+00, -4.3025e+00, -3.5938e+00, -6.4468e+00, -5.4404e+00,
         -5.8367e+00, -9.3099e-01, -5.8367e+00, -1.5461e-01, -7.4089e+00,
         -2.9910e-01, -7.4441e-02, -4.3025e+00, -1.5461e-01, -3.8457e+00,
         -1.5461e-01, -4.4528e+00, -7.4441e-02, -3.8139e+00, -3.8457e+00,
         -3.8139e+00, -3.8139e+00, -3.0649e-01, -3.6748e-01, -3.7105e-02,
         -5.8367e+00, -3.8457e+00, -4.4528e+00, -1.9898e+00, -6.1269e+00,
         -4.4528e+00, -1.5461e-01, -7.4089e+00, -2.5398e-02],
        [-4.6556e-03, -2.1488e-02, -3.7028e-02, -3.7292e+00, -1.4292e-02,
         -5.3826e-01, -4.6556e-03, -9.9787e-01, -1

tensor([[-4.1005, -1.4663,  5.8797]], grad_fn=<AddmmBackward0>)
Premise: Two women are embracing while holding to go packages.
Hypothesis: The men are fighting outside a deli.
Entailment: 4.627431553672068e-05
Neutral: 0.0006447185878641903
Contradiction: 0.9993090629577637
